In [2]:
import pyspark 
from pyspark.sql import SparkSession,Row,DataFrame
import pyspark.sql.functions as F
from pyspark.sql.types import *


spark=SparkSession.builder.master("local[1]")\
        .appName("movie_local")\
        .config("spark.driver.extraClassPath","/Users/eduardoalberto/opt/spark-3.5.3/jars/mysql-connector-j-9.1.0.jar")\
        .config("spark.sql.shuffle.partitions", "7") \
        .config("spark.sql.warehouse.dir", "/Users/eduardoalberto/LoadFile/repository/database/") \
        .config("spark.dynamicAllocation.enabled", "true") \
        .config("spark.executor.cores", "4") \
        .config("spark.dynamicAllocation.minExecutors", "1") \
        .config("spark.dynamicAllocation.maxExecutors", "5") \
        .enableHiveSupport()\
        .getOrCreate()
sc = spark.sparkContext
spark.sparkContext.setLogLevel("OFF") 
print('PySpark Version :'+spark.version)
print('PySpark Version :'+spark.sparkContext.version)

spark

25/07/20 21:17:20 WARN Utils: Your hostname, MacBook-Pro-de-Eduardo.local resolves to a loopback address: 127.0.0.1; using 192.168.3.180 instead (on interface en0)
25/07/20 21:17:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/20 21:17:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


PySpark Version :3.5.4
PySpark Version :3.5.4


In [1]:
! ls /Users/eduardoalberto/LoadFile/data_2022*.csv



zsh:1: no matches found: /Users/eduardoalberto/LoadFile/data_2022*.csv


In [4]:
spark.sql(" CREATE DATABASE IF NOT EXISTS dbdev150").show()


++
||
++
++



In [5]:
df = spark.read.option("delimiter", "\t").option("header", True).csv('/Users/eduardoalberto/LoadFile/data_2022*.csv')

df.show(truncate=False)



AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/Users/eduardoalberto/LoadFile/data_2022*.csv.

In [ ]:
import pyspark 
from pyspark.sql import SparkSession,Row,DataFrame
import pyspark.sql.functions as F
from pyspark.sql.types import *

def fn_ingest (df: DataFrame) -> DataFrame :

    dfs = df.filter(" deathYear <> r'\\N' and deathYear <> r'\\N'  ")\
            .withColumn('profession_array',F.explode((F.split(F.col("primaryProfession"), ","))))\
            .withColumn('knownForTitles_array',F.explode((F.split(F.col("knownForTitles"),","))))\
            .drop_duplicates(['profession_array','knownForTitles_array'])

    
    dff = dfs.filter("profession_array <> r'\\N' and knownForTitles_array <> r'\\N'")
  
    return dff
tb = fn_ingest(df)

tb.limit(50000).write.mode("overwrite").saveAsTable("dbdev150.movie")


#tb.limit(50000).write.mode("overwrite").jdbc("jdbc:mysql://localhost:3306/myDbUser", "myDbUser.movie",properties={"user": "root", "password": "mysql"})

#104.466.466
#3.449.518


In [ ]:
df = spark.table("dbdev150.movie")

df.write.mode("overwrite").parquet("/Users/eduardoalberto/LoadFile/parquet/movie")



dfs = spark.read.option("delimiter", ";").option("header", True).csv("/Users/eduardoalberto/LoadFile/part-00000-055103f0-b275-4e27-b667-0c2c25d0636a-c000.csv")
dfs.write.partitionBy("dt_release").mode("overwrite").parquet("/Users/eduardoalberto/LoadFile/parquet/movie02")
# df.write.parquet("/Users/eduardoalberto/LoadFile/parquet/arqui02/")

df.show()

dfs.show()




In [3]:
# temp = spark.read.parquet("/Users/eduardoalberto/LoadFile/parquet/movie02")
# temp.createOrReplaceTempView("paquteTb")
# spark.sql("drop table if exists dbdev150.tb_movie02")
# spark.sql("create table  dbdev150.tb_movie02 as select * from paquteTb ")
spark.sql("select * from  dbdev150.tb_movie02").show()


+-----+-----+--------------------+--------------------+--------------------+--------+---------+-----------------+--------------------+--------------------+----------+--------------------+--------+-------+--------+--------------------+--------------------+-----+------------+----------+-----+-----------+----------+
| id01|   id|                name|         poster_path|       backdrop_path|  budget|  imdb_id|original_language|      original_title|            overview|popularity|         path_poster| revenue|runtime|  status|             tagline|               title|video|vote_average|vote_count| id02|name_geners|dt_release|
+-----+-----+--------------------+--------------------+--------------------+--------+---------+-----------------+--------------------+--------------------+----------+--------------------+--------+-------+--------+--------------------+--------------------+-----+------------+----------+-----+-----------+----------+
|10402|95664|Deuce Bigalow Col...|/szTJbYJnm3OPwYxQ...|